In [ ]:
import sys
sys.path.append("../")
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [ ]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [ ]:
df_raw.shape

In [ ]:
# gets the last 6000 and creates a copy
df_anal = df_raw.copy() # df_raw.iloc[-6000:].copy()
df_anal.shape

In [ ]:
# put the patterns and the RSI onto the dataframe
df_anal = RSI(df_anal)

In [ ]:
df_anal.head()

In [ ]:
# because RSI starts at 0 this will not work as our data index does not start there, so RSI is not working
# what we do is
df_anal.reset_index(drop=True, inplace=True)
df_anal = RSI(df_anal)
df_anal.head()

In [ ]:
# RSI is 14 periouds so we look at tail
df_anal.tail()

In [ ]:
# to get the candle pattern  can see candle types 
df_anal = apply_patterns(df_anal)
df_anal.tail()

In [ ]:
# apply the MA to the data
df_anal['EMA_200'] = df_anal.mid_c.ewm(span=200, min_periods=200).mean()


In [ ]:
# CHECK WHAT COLUMNS WE HAVE
df_anal.columns

In [ ]:
# grab a subset pf columns
the_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [ ]:
# make df_slim dataframe
df_slim = df_anal[the_cols].copy()

In [ ]:
df_slim.head(500)

In [ ]:
# to get rid of the NaN due to not enought periods
df_slim.dropna(inplace=True)
# do a index reset
df_slim.reset_index(drop=True, inplace=True)


In [ ]:
df_slim.head(500)

In [ ]:
# Strategy
# Bullish engulfing candle
# price is above the 200 EMA
# RSI above 50
# enter on candle close
# stop at bottom of body
# TP is 1.5 the stop

BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def trade_signal(row):
    # Bullish engulfing candle
    if row.ENGULFING == True:
        # Buy setting price aboce EMA
        if row.direction == BUY and row.mid_l > row.EMA_200:
            # check RSI
            if row.RSI_14 > RSI_LIMIT:
                return BUY
# Sell order
        if row.direction == SELL and row.mid_h < row.EMA_200:
            # check RSI
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE            

In [ ]:
# apply our function to the dataframe
df_slim["SIGNAL"] = df_slim.apply(trade_signal, axis=1)

In [ ]:
# to see how many signals we have
df_slim["SIGNAL"].value_counts()

In [ ]:
# apply take profit and stop
RISK = 1.0
REWARD = 1.5
def take_profit(row):
    if row.SIGNAL != NONE:
        return(row.mid_c - row.mid_o) * REWARD + row.mid_c
    else:
        return(0.0)

def stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return(0.0)
    

In [ ]:
df_slim["TAKE_PROFIT"] = df_slim.apply(take_profit, axis=1)
df_slim["STOP_LOSS"] = df_slim.apply(stop_loss, axis=1)

In [ ]:
df_slim[df_slim.SIGNAL == BUY].head()

In [ ]:
df_plot = df_slim.iloc[70:104]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[df_plot.SIGNAL != NONE]

markers = ["mid_c", "TAKE_PROFIT", "STOP_LOSS"]

marker_colors = ['blue', 'green', 'red'] 

In [ ]:
for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker = dict(color=marker_colors[i], size=12)    
    ))
cp.show_plot(line_traces=["EMA_200"])    

In [ ]:
# df_results.head()
# df_results[df_results.SIGNAL == BUY].head()

In [ ]:

class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c 
        self.SIGNAL = row.SIGNAL
        self.TP = row.TAKE_PROFIT
        self.SL = row.STOP_LOSS
        self.result = 0.0
        self.start_time = row.time
        self.end_time = row.time
        self.duration = 0
        

    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        

    def update(self, row):
        self.duration +=1
        
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, REWARD, row.mid_h)
            elif row.mid_l <= self.SL:  
                self.close_trade(row, RISK, row.mid_l)


        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, REWARD, row.mid_l)
            elif row.mid_h >= self.SL:  
                self.close_trade(row, RISK, row.mid_h)

In [ ]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]        

    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [ ]:
len(open_trades)

In [ ]:
len(closed_trades)

In [ ]:
vars(closed_trades[0])

In [ ]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [ ]:
df_results

In [ ]:
df_results.result.sum()

In [ ]:
df_results.sort_values(by="start_index", inplace=True)

In [ ]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [ ]:
df_m5.shape

In [ ]:
df_m5.time.max()

In [ ]:
df_raw.time.max()

In [ ]:
from dateutil import parser

In [ ]:
time_min = parser.parse('2021-12-15T10:00:00Z')
time_max = parser.parse('2021-12-15T11:00:00Z')

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
df_m5_start = df_m5[(df_m5.time >= time_min) & (df_m5.time <= time_max)] 
df_raw_start = df_raw[(df_raw.time >= time_min) & (df_raw.time <= time_max)] 



In [ ]:
df_raw_start 

In [ ]:
df_m5_slim = df_m5[['time', 'mid_h', 'mid_l']].copy()

In [ ]:
df_m5_slim

In [ ]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [ ]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [ ]:
df_signals['m5_start']

In [ ]:
df_signals['start_index_h1'] = df_signals.index

In [ ]:
df_signals.head()

In [ ]:
df_signals.columns

In [ ]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'bid_c', 'ask_c', 'ENGULFING',  'EMA_200', 'RSI_14', 'direction'], inplace=True, axis=1)

In [ ]:
df_signals.head()

In [ ]:
df_signals.rename(columns=dict(
    mid_c = 'start_price',
    m5_start = 'time'  
), inplace=True)

In [ ]:
df_signals.head(2)



In [ ]:

df_m5_slim.head(2)

In [ ]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [ ]:
merged.fillna(0, inplace=True)

In [ ]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.SIGNAL = merged.start_index_h1.astype(int)

In [ ]:
merged.head()

In [ ]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TAKE_PROFIT
        self.SL = row.STOP_LOSS
        self.result = 0.0
        self.start_time = row.time
        self.end_time = row.time
        self.duration = 0
        

    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        

    def update(self, row):
        self.duration +=1
        
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, REWARD, row.mid_h)
            elif row.mid_l <= self.SL:  
                self.close_trade(row, RISK, row.mid_l)


        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, REWARD, row.mid_l)
            elif row.mid_h >= self.SL:  
                self.close_trade(row, RISK, row.mid_h)
        

In [ ]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]        



In [27]:
len(closed_trades_m5)

NameError: name 'closed_trades_m5' is not defined